# Combined dispatch and price plots
Combined prices and dispatch plots, created for paper on *pommesdispatch* to show interrelation between dispatch and price situations.

## Package imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import OrderedDict
from pommesevaluation.price_validation import (
    read_and_reshape_historical_prices
)

from pommesevaluation.combined_plots import plot_combined_dispatch_and_price_plot

plt.rcParams.update({'font.size': 12})

## Read in data and preprocess
Data read in:
* Dispatch Results
* Power prices
* Generation potential for RES (to evaluate curtailment)

Preprocessing:
* Filter results for German electricity bus
* Combine historical and model prices

In [ ]:
# Define situation to analyze
simulation_year = 2017
start_time_step = f"{simulation_year}-04-30 00:00:00"
end_time_step = f"{simulation_year}-05-01 23:00:00"

# Define, where results are stored and can be retireved
path_results = "./model_results/"
path_inputs = "./model_inputs/"
path_plots = "./plots/"
path_historical_prices = "./data/prices/"

dispatch_results_file_name = f"dispatch_LP_start-{simulation_year}-01-01_364-days_simple_complete_production_UPDATE.csv"
price_results_file_name = f"dispatch_LP_start-{simulation_year}-01-01_364-days_simple_complete_power-prices_UPDATE.csv"

sources_res_file_name = f"sources_renewables_{simulation_year}.csv"
transformers_res_file_name = f"transformers_renewables_{simulation_year}.csv"
sources_res_ts_file_name = f"sources_renewables_ts_{simulation_year}.csv"

In [ ]:
# Dispatch results
buses_el = pd.read_csv(path_results + dispatch_results_file_name, index_col=0)
# Filter generation, exports and imports for Germany
bus_DE = buses_el[[col for col in buses_el.columns if "DE" in col]]

# Power prices
model_prices = pd.read_csv(
    path_results + price_results_file_name, 
    sep=",",
    decimal=".",
    index_col=0,
    parse_dates=True
)
if simulation_year < 2022:
    historical_prices = read_and_reshape_historical_prices(
        2017, 
        f"{path_historical_prices}auction_spot_prices_germany_austria_2017.csv"
    )
    power_prices = pd.concat([historical_prices, model_prices], axis=1)
else:
    power_prices = model_prices.copy()
power_prices.rename(columns={"historical_price": "historical price", "Power price": "model price"}, inplace=True)

# RES infeed potential (Extract values for Germany)
sources_res = pd.read_csv(path_inputs + sources_res_file_name, index_col=0)
sources_res = sources_res.loc[sources_res["country"] == "DE"]
transformers_res = pd.read_csv(path_inputs + transformers_res_file_name, index_col=0)
sources_res_ts = pd.read_csv(path_inputs + sources_res_ts_file_name, index_col=0, parse_dates=True)
sources_res_ts = sources_res_ts[[col for col in sources_res_ts.columns if col.split("_")[0] == "DE"]]

# Combine data sets (fluctuating + nonfluctuating capacities)
transformers_res = transformers_res.groupby("from")[["capacity"]].sum()
sources_res = pd.concat([sources_res, transformers_res])
sources_res_ts_abs = sources_res_ts.mul(sources_res["capacity"])
sources_res_ts_abs.columns = [col.split("_")[-1] for col in sources_res_ts_abs.columns]

## Extract necessary dispatch information
For a detailled description, see dedicated notebook for dispatch validation.
* Import / exports 
* storage inflow / outflow
* generation

### Imports & exports

In [ ]:
# Filter imports and exports and calculate overall and net imports & exports
im_ex_DE = bus_DE[[col for col in bus_DE.columns if "link_" in col]].copy()
export_links = [col for col in im_ex_DE if "('DE_link_" in col]
import_links = [col for col in im_ex_DE if "DE_bus_el')" in col]
im_ex_DE["overall_exports"] = im_ex_DE[export_links].sum(axis=1)
im_ex_DE["overall_imports"] = -im_ex_DE[import_links].sum(axis=1)
im_ex_DE["net_export"] = im_ex_DE["overall_exports"] + im_ex_DE["overall_imports"]
im_ex_DE.index = pd.to_datetime(im_ex_DE.index)

# Formatting
im_ex_plot = im_ex_DE.copy()
im_ex_plot[import_links] *= -1
imports = {
    key: f"import_{key[3:5]}" 
    for key in import_links 
    if "DK" not in key and "SE" not in key
}
for key in import_links:
    if "DK" in key or "SE" in key:
        imports[key] = f"import_{key[3:6]}"

exports = {
    key: f"export_{key[11:13]}" 
    for key in export_links
    if "DK" not in key and "SE" not in key
}
for key in export_links:
    if "DK" in key or "SE" in key:
        exports[key] = f"export_{key[11:14]}"

im_ex_plot.rename(columns=imports, inplace=True)
im_ex_plot.rename(columns=exports, inplace=True)
im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot.columns 
        if col not in imports.values() and col not in exports.values()
    ], 
    inplace=True
)

# Color codes
country_colors = {
    "FR": "blue",
    "CH": "red",
    "BE": "black",
    "CZ": "gray",
    "DK1": "lightblue",
    "DK2": "turquoise",
    "PL": "dimgray",
    "NL": "orange",
    "NO": "green",
    "SE4": "yellow",
    "AT": "purple"
}
colors_im = {
    f"import_{key}": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
}
colors_ex = {
    f"export_{key}": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
}
country_colors_im_ex = {**colors_im, **colors_ex}

# Remove exchange with Austria before market splitting in 2018
if simulation_year <= 2018:
    im_ex_plot.drop(columns=["export_AT", "import_AT"], inplace=True)
    country_colors.pop("AT")
    country_colors_im_ex.pop("export_AT")
    country_colors_im_ex.pop("import_AT")

for country in country_colors.keys():
    im_ex_plot[f"net_export_{country}"] = (
        im_ex_plot[f"export_{country}"]
        + im_ex_plot[f"import_{country}"]
    )
im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot.columns
        if "net_export" not in col
    ],
    inplace=True
)

# Directions
for col in im_ex_plot.columns:
    im_ex_plot[f"{col}_pos"] = np.where(im_ex_plot[col] >= 0, im_ex_plot[col], 0)
    im_ex_plot[f"{col}_neg"] = np.where(im_ex_plot[col] < 0, im_ex_plot[col], 0)    

im_ex_plot = im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot
        if "pos" not in col and "neg" not in col
    ]
)

# Overall data set
im_ex_overall = im_ex_plot.copy()
im_ex_overall["overall_net_export"] = im_ex_overall.sum(axis=1)
im_ex_overall["overall_net_export_pos"] = np.where(
    im_ex_overall["overall_net_export"] >= 0,   
    im_ex_overall["overall_net_export"], 
    0
)
im_ex_overall["overall_net_export_neg"] = np.where(
    im_ex_overall["overall_net_export"] < 0, 
    im_ex_overall["overall_net_export"], 
    0
)

### Storage operation

In [ ]:
storage_DE = bus_DE[[col for col in bus_DE.columns if "storage" in col]]
storage_DE_renamed = storage_DE.rename(columns={
    "(('DE_bus_el', 'DE_storage_el_PHS'), 'flow')": "storage_in",
    "(('DE_storage_el_PHS', 'DE_bus_el'), 'flow')": "storage_out"
})
storage_DE_renamed["storage_in"] *= -1
storage_DE_renamed["net_storage"] = storage_DE_renamed["storage_out"] + storage_DE_renamed["storage_in"]
storage_DE_renamed.index = pd.to_datetime(storage_DE_renamed.index)

### Generation
> _Note: Generation includes storage outflows_

In [ ]:
# Aggregate all generation units
power_generation_DE = bus_DE[[
    col for col in bus_DE.columns 
    if col not in im_ex_DE.columns
    and col not in storage_DE.columns
]]

# Drop demand, power prices and shortage
power_generation_DE = power_generation_DE.drop(
    columns=[
        "(('DE_bus_el', 'DE_sink_el_load'), 'flow')", 
        "(('DE_bus_el', 'None'), 'duals')",
        "(('DE_source_el_shortage', 'DE_bus_el'), 'flow')",
    ]
)

power_generation_DE["overall_generation"] = power_generation_DE.sum(axis=1)
power_generation_DE.index = pd.to_datetime(power_generation_DE.index)

# Renaming
fuel_dict = {
    'ROR': 'Wasser',
    'biomass': 'Biomasse',
    'biomassEEG': 'Biomasse',
    'landfillgas': 'Deponiegas',
    'geothermal': 'Geothermie',
    'minegas': 'Grubengas',
    'larga': 'Klärgas',
    'windonshore': 'Windenergie an Land',
    'windoffshore': 'Windenergie auf See',
    'solarPV': 'Solare Strahlungsenergie',
    'uranium': 'Kernenergie',
    'lignite': 'Braunkohle',
    'hardcoal': 'Steinkohle',
    'waste': 'Abfall',
    'natgas': 'Erdgas',
    'otherfossil': 'Andere fossile',
    'mixedfuels': 'Mehrere fossile',
    'oil': 'Heizöl',
    'hydrogen': 'Wasserstoff',
}

# Color codes
colors = {
    'solarPV': '#fcb001',
    'windonshore': '#82cafc',
    'windoffshore': '#0504aa',
    'uranium': '#e50000',
    'lignite': '#7f2b0a',
    'otherfossil': '#d8dcd6',
    'hardcoal': '#000000',
    'waste': '#c04e01',
    'mixedfuels': '#a57e52',
    'biomass': '#15b01a',
    'geothermal': '#ff474c',
    'otherres': '#06c2ac',
    'minegas': '#650021',
    'natgas': '#929591',
    'oil': '#aaa662',
    'ROR': '#c79fef',
    "storage_out" : 'darkblue',
    'hydrogen': '#6fa8dc',
}

# Group outputs by energy carrier
energy_sources_dict = OrderedDict()

for fuel in fuel_dict.keys():
    energy_sources_dict[fuel] = [
        entry for entry in power_generation_DE.columns.values if fuel in entry
    ]   
    
# Store the aggregated production results per energy source
generation = pd.DataFrame()
for key, val in energy_sources_dict.items():
    generation[key] = power_generation_DE[val].sum(axis = 1)

# Aggregate
generation["biomass"] = generation["biomass"] + generation["biomassEEG"]
generation["otherres"] = (
    generation["landfillgas"]
    + generation["larga"]
)

generation.drop(
    columns=[
        col for col in generation.columns
        if col not in colors.keys()
    ],
    inplace=True
)

generation["storage_out"] = storage_DE_renamed["storage_out"]

## Evaluate dispatch and power prices for certain time frame

In [ ]:
overall_situation = pd.concat(
    [
        storage_DE_renamed.loc[start_time_step:end_time_step, "storage_in"],
        generation.loc[start_time_step:end_time_step],
        im_ex_overall.loc[start_time_step:end_time_step, ["overall_net_export_pos", "overall_net_export_neg"]]
    ],
    axis = 1
).round(3)

overall_situtation_colors = {
    **colors,
    "storage_in": "darkblue", 
    "overall_net_export_pos": "#ffefef",
    "overall_net_export_neg": "#ffefef"
}

power_prices_slice = power_prices.loc[start_time_step:end_time_step]

power_prices_colors = {
    "historical price": "#004aff",
    "model price": "#ff461f",
}

color = {
    "dispatch": overall_situtation_colors,
    "power_prices": power_prices_colors
}

## Create combined plot
Use a function for reusability

### Create plot neglecting RES curtailment

In [ ]:
plot_combined_dispatch_and_price_plot(
    start_time_step, 
    end_time_step, 
    simulation_year, 
    overall_situation, 
    power_prices_slice, 
    color, 
    figsize=(15, 10),
    dispatch_limits={"bottom": -20000, "top": 120000},
    price_limits={"bottom": -100, "top": 40},
    path_plots=path_plots
)

### Evaluate RES curtailment
RES curtailment per energy carrier is the difference between the feed-in-potential and realised generation

In [ ]:
sources_res_ts_abs_slice = sources_res_ts_abs.loc[start_time_step:end_time_step]

# Evaluate difference in RES dispatch (curtailment; given with a negative sign)
realised_res = overall_situation[[col for col in sources_res_ts_abs_slice if col in overall_situation.columns]]
res_potential = sources_res_ts_abs_slice[[col for col in realised_res.columns]]

res_curtailed = (realised_res - res_potential).round(2)
res_curtailed.columns = [col + "_curtailment" for col in res_curtailed.columns]
res_curtailed = res_curtailed[[col for col in res_curtailed if res_curtailed[col].sum() != 0]]

### Create plot including RES curtailment

In [ ]:
overall_situation_incl_curtailment = pd.concat(
    [res_curtailed, overall_situation], 
    axis= 1
)

In [ ]:
res_curtailed_colors = {key + "_curtailment": value for key, value in colors.items() if key in realised_res.columns}

color["dispatch"] = {**res_curtailed_colors, **color["dispatch"]}

plot_combined_dispatch_and_price_plot(
    start_time_step, 
    end_time_step, 
    simulation_year, 
    overall_situation_incl_curtailment, 
    power_prices_slice, 
    color, 
    figsize=(15, 10),
    dispatch_limits={"bottom": -20000, "top": 120000},
    price_limits={"bottom": -100, "top": 40},
    path_plots=path_plots,
    file_name_suffix="_w_curtailment"
)